###Kaggle competition
https://inclass.kaggle.com/c/model-t4/data

Predict CPU load based on applications running on a server cluster.

This competition is part of a machine learning workshop given at InTraffic.

The goal is to predict the load on the CPUs in a cluster of servers based on the behavior of a series of applications running on these servers.

There are two CPUs in each server. There are seven servers in the cluster. The prediction is for the second CPU.

The dataset consists of a set of variables that were measured over about a one month period. Measurements were taken in one minute intervals and on each server. Measurements are usually the average or sum over that one minute interval. For instance the number of packets received, the average number of IO operations, etc

The set is data from a real cluster that is used to control train traffic in a geographical area spanning several cities.

Data was collected with a tool called T4 (Total Timeline Tracking Tool).

###Data fields

* sample time - the date and time the data was sampled.
* m_id - the ID of the server the data was sampled at.
* appxxxx - data about specific application.
* pagexxx - data on memory usage of the server.
* syst_xxx - data on page fault rate, number of processes, etc.
* state_xxx - data on the state the system is in.
* io_xxx - data about general IO usage, (file IO, direct IO).
* tcp_xxx - data on incoming and outgoing TCP traffic.
* llxxx, ewxxx - data on incoming and outgoing network traffic.
* cpu_01_busy - the variable we are trying to predict.

###EDA

In [19]:
import pandas as pd
import numpy as np
import pdb

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sample_soln = pd.read_csv('sampleSolution.csv')

In [5]:
df_train.columns

Index([u'sample_time', u'm_id', u'syst_direct_ipo_rate',
       u'syst_buffered_ipo_rate', u'syst_page_fault_rate',
       u'syst_page_read_ipo_rate', u'syst_process_count', u'syst_other_states',
       u'page_page_write_ipo_rate', u'page_global_valid_fault_rate',
       u'page_free_list_size', u'page_modified_list_size',
       u'io_mailbox_write_rate', u'io_split_transfer_rate',
       u'io_file_open_rate', u'io_logical_name_trans', u'io_page_reads',
       u'io_page_writes', u'page_free_list_faults',
       u'page_modified_list_faults', u'page_demand_zero_faults',
       u'state_compute', u'state_mwait', u'state_lef', u'state_hib',
       u'state_cur', u'app06_dirio', u'app06_bufio', u'app06_pgflts',
       u'app06_proccount', u'app06_pagesgbl', u'app06_pagesproc',
       u'app07_dirio', u'app07_bufio', u'app07_pgflts', u'app07_proccount',
       u'app07_pagesgbl', u'app07_pagesproc', u'app04_dirio', u'app04_bufio',
       u'app04_pgflts', u'app04_proccount', u'app04_pagesgbl',
    

In [8]:
# check for missing data
for column in df_train.columns:
    if sum(df_train[column].notnull()) == df_train.shape[0]:
        print '%s has no missing rows' % column
    else:
        print '%s has %d rows' % (column, df_train.shape[0] - sum(df[column].notnull()))
        
# no missing data

sample_time has no missing rows
m_id has no missing rows
syst_direct_ipo_rate has no missing rows
syst_buffered_ipo_rate has no missing rows
syst_page_fault_rate has no missing rows
syst_page_read_ipo_rate has no missing rows
syst_process_count has no missing rows
syst_other_states has no missing rows
page_page_write_ipo_rate has no missing rows
page_global_valid_fault_rate has no missing rows
page_free_list_size has no missing rows
page_modified_list_size has no missing rows
io_mailbox_write_rate has no missing rows
io_split_transfer_rate has no missing rows
io_file_open_rate has no missing rows
io_logical_name_trans has no missing rows
io_page_reads has no missing rows
io_page_writes has no missing rows
page_free_list_faults has no missing rows
page_modified_list_faults has no missing rows
page_demand_zero_faults has no missing rows
state_compute has no missing rows
state_mwait has no missing rows
state_lef has no missing rows
state_hib has no missing rows
state_cur has no missing ro

In [64]:
# check categorical vs. continuous features
for n, t in zip(df_train.columns, df_train.dtypes): 
    print n, t 

sample_time object
m_id object
syst_direct_ipo_rate float64
syst_buffered_ipo_rate float64
syst_page_fault_rate float64
syst_page_read_ipo_rate float64
syst_process_count int64
syst_other_states int64
page_page_write_ipo_rate float64
page_global_valid_fault_rate float64
page_free_list_size int64
page_modified_list_size int64
io_mailbox_write_rate float64
io_split_transfer_rate float64
io_file_open_rate float64
io_logical_name_trans float64
io_page_reads float64
io_page_writes float64
page_free_list_faults float64
page_modified_list_faults float64
page_demand_zero_faults float64
state_compute int64
state_mwait int64
state_lef int64
state_hib int64
state_cur int64
app06_dirio int64
app06_bufio int64
app06_pgflts int64
app06_proccount int64
app06_pagesgbl int64
app06_pagesproc int64
app07_dirio float64
app07_bufio float64
app07_pgflts float64
app07_proccount int64
app07_pagesgbl int64
app07_pagesproc int64
app04_dirio float64
app04_bufio float64
app04_pgflts float64
app04_proccount int64


In [47]:
# create datetime obj and unix timestamp from string timestamp
from datetime import datetime

df_train['sample_time_dt_obj'] = df_train['sample_time'].apply(\
                                 lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df_train['sample_time_unix'] = df_train['sample_time_dt_obj'].apply(\
                               lambda x: (x - datetime(1970,1,1)).total_seconds())

In [48]:
df_train.columns

Index([u'sample_time', u'm_id', u'syst_direct_ipo_rate',
       u'syst_buffered_ipo_rate', u'syst_page_fault_rate',
       u'syst_page_read_ipo_rate', u'syst_process_count', u'syst_other_states',
       u'page_page_write_ipo_rate', u'page_global_valid_fault_rate',
       u'page_free_list_size', u'page_modified_list_size',
       u'io_mailbox_write_rate', u'io_split_transfer_rate',
       u'io_file_open_rate', u'io_logical_name_trans', u'io_page_reads',
       u'io_page_writes', u'page_free_list_faults',
       u'page_modified_list_faults', u'page_demand_zero_faults',
       u'state_compute', u'state_mwait', u'state_lef', u'state_hib',
       u'state_cur', u'app06_dirio', u'app06_bufio', u'app06_pgflts',
       u'app06_proccount', u'app06_pagesgbl', u'app06_pagesproc',
       u'app07_dirio', u'app07_bufio', u'app07_pgflts', u'app07_proccount',
       u'app07_pagesgbl', u'app07_pagesproc', u'app04_dirio', u'app04_bufio',
       u'app04_pgflts', u'app04_proccount', u'app04_pagesgbl',
    

In [63]:
# convert categorical variable m_id to numeric
df_train_clean = pd.concat([df_train, pd.get_dummies(df_train['m_id']).drop('g', axis=1)], 
                           axis=1)

In [65]:
# check categorical vs. continuous features
for n, t in zip(df_train_clean.columns, df_train_clean.dtypes): 
    print n, t 

sample_time object
m_id object
syst_direct_ipo_rate float64
syst_buffered_ipo_rate float64
syst_page_fault_rate float64
syst_page_read_ipo_rate float64
syst_process_count int64
syst_other_states int64
page_page_write_ipo_rate float64
page_global_valid_fault_rate float64
page_free_list_size int64
page_modified_list_size int64
io_mailbox_write_rate float64
io_split_transfer_rate float64
io_file_open_rate float64
io_logical_name_trans float64
io_page_reads float64
io_page_writes float64
page_free_list_faults float64
page_modified_list_faults float64
page_demand_zero_faults float64
state_compute int64
state_mwait int64
state_lef int64
state_hib int64
state_cur int64
app06_dirio int64
app06_bufio int64
app06_pgflts int64
app06_proccount int64
app06_pagesgbl int64
app06_pagesproc int64
app07_dirio float64
app07_bufio float64
app07_pgflts float64
app07_proccount int64
app07_pagesgbl int64
app07_pagesproc int64
app04_dirio float64
app04_bufio float64
app04_pgflts float64
app04_proccount int64


In [67]:
df_train_numeric = df_train_clean.drop(['sample_time_dt_obj', 'm_id', 'sample_time'], axis=1)

In [68]:
# check categorical vs. continuous features
for n, t in zip(df_train_numeric.columns, df_train_numeric.dtypes): 
    print n, t 

syst_direct_ipo_rate float64
syst_buffered_ipo_rate float64
syst_page_fault_rate float64
syst_page_read_ipo_rate float64
syst_process_count int64
syst_other_states int64
page_page_write_ipo_rate float64
page_global_valid_fault_rate float64
page_free_list_size int64
page_modified_list_size int64
io_mailbox_write_rate float64
io_split_transfer_rate float64
io_file_open_rate float64
io_logical_name_trans float64
io_page_reads float64
io_page_writes float64
page_free_list_faults float64
page_modified_list_faults float64
page_demand_zero_faults float64
state_compute int64
state_mwait int64
state_lef int64
state_hib int64
state_cur int64
app06_dirio int64
app06_bufio int64
app06_pgflts int64
app06_proccount int64
app06_pagesgbl int64
app06_pagesproc int64
app07_dirio float64
app07_bufio float64
app07_pgflts float64
app07_proccount int64
app07_pagesgbl int64
app07_pagesproc int64
app04_dirio float64
app04_bufio float64
app04_pgflts float64
app04_proccount int64
app04_pagesgbl int64
app04_page

In [17]:
# compute VIF to test for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [84]:
# some columns are all 0, hence VIF is not possible
import math
math.isnan(variance_inflation_factor(df_train_numeric.as_matrix(), 24))

True

In [81]:
df_train_numeric['app06_bufio'].describe()

count    178780
mean          0
std           0
min           0
25%           0
50%           0
75%           0
max           0
Name: app06_bufio, dtype: float64

In [87]:
vif_score = dict()

In [88]:
for col_index, column in enumerate(df_train_numeric.columns):
    vif = variance_inflation_factor(df_train_numeric.as_matrix(), col_index)
    vif_score[column] = vif
    if vif >= 30:
        print 'column: %s may have a multicollinearity problem. VIF = %d' % (column, vif)
    elif math.isnan(vif):
        print 'column: VIF of %s cannot be calculated' % column
    else:
        print 'column: %s is good. VIF = %d' % (column, vif)

column: syst_direct_ipo_rate is good. VIF = 3
column: syst_buffered_ipo_rate is good. VIF = 2
column: syst_page_fault_rate may have a multicollinearity problem. VIF = 33449
column: syst_page_read_ipo_rate may have a multicollinearity problem. VIF = 4047
column: syst_process_count may have a multicollinearity problem. VIF = 8630
column: syst_other_states may have a multicollinearity problem. VIF = 1589
column: page_page_write_ipo_rate may have a multicollinearity problem. VIF = 41
column: page_global_valid_fault_rate may have a multicollinearity problem. VIF = 3219
column: page_free_list_size may have a multicollinearity problem. VIF = 23370
column: page_modified_list_size may have a multicollinearity problem. VIF = 7680
column: io_mailbox_write_rate is good. VIF = 4
column: io_split_transfer_rate is good. VIF = 1
column: io_file_open_rate is good. VIF = 1
column: io_logical_name_trans is good. VIF = 2
column: io_page_reads may have a multicollinearity problem. VIF = 1068
column: io_pag

KeyboardInterrupt: 

In [ ]:
# split features into categories
import re
m = re.match('app', 'ex01_2321')
m.group()

In [ ]:
appxxxx - data about specific application.
pagexxx - data on memory usage of the server.
syst_xxx - data on page fault rate, number of processes, etc.
state_xxx - data on the state the system is in.
io_xxx - data about general IO usage, (file IO, direct IO).
tcp_xxx - data on incoming and outgoing TCP traffic.
llxxx, ewxxx - data on incoming and outgoing network traffic